In [ ]:
from openeye import oedocking
from openeye import oeomega
from openeye import oechem

In [ ]:
# Load the T4 receptor; the files of the different receptors used in this study can be found in:
# T4L-temperature-effects/Docking/OEdock/binders-non-binders/files-to-prepare-receptors/cryo-closed-raw 
imstr = oemolistream('receptor.pdb')
protein = oechem.OEGraphMol()
oechem.OEReadMolecule(imstr, protein)
#imstr.close()

# Load a reference ligand to specify the binding site
ligand = oechem.OEGraphMol()
imstr = oechem.oemolistream('toluene_oe.mol2')
oechem.OEReadMolecule(imstr, ligand)
imstr.close()

# Initialize the receptor for docking
receptor = oechem.OEGraphMol()
oedocking.OEMakeReceptor(receptor, protein, ligand)

In [ ]:
# Set the docking method and resolution 
dock_method = oedocking.OEDockMethod_Chemscore
dock_resolution = oedocking.OESearchResolution_Default
sdtag = oedocking.OEDockMethodGetName( dock_method )

# OEDocking object
dock = oedocking.OEDock( dock_method, dock_resolution)

if not dock.Initialize(receptor):
    raise Exception("Unable to initialize Docking with {0}".format(self.args.receptor))

In [ ]:
def dock_molecule( dock: "OEDock", sdtag: str, num_poses: int, mcmol ) -> tuple:
    ''' Docks the multiconfomer molecule, with the given number of poses
        Returns a tuple of the docked molecule (dockedMol) and its score
        i.e. ( dockedMol, score )
    '''
    dockedMol = oechem.OEMol()

    # Dock the molecule
    res = dock.DockMultiConformerMolecule(dockedMol, mcmol, num_poses)
    
    if res == oedocking.OEDockingReturnCode_Success:
        
        # Label the molecule with the score and SDTag
        oedocking.OESetSDScore(dockedMol, dock, sdtag)
        dock.AnnotatePose(dockedMol)
        score = dock.ScoreLigand(dockedMol)
        oechem.OESetSDData(dockedMol, sdtag, "{}".format(score))
        return dockedMol, score
    
    else:
        # raise an exception if the docking is not successful
        raise Exception("Unable to dock ligand {0} to receptor".format( dockedMol ))

In [ ]:
# Decoys created via DUD-E
decoys = open("decoys.smi").read().splitlines()

# Zinc list - described as binders
zinc = open("zinc.smi").read().splitlines()

# Experimentally validated active compounds - Mobley work + Minh et al 
actives = open("mobley-minh-actives.smi").read().splitlines()


#combine all the compounds in one list
all_compounds = decoys + zinc + actives

In [ ]:
# save list to file 
with open('all_compounds.smi', 'w') as f:
    for item in all_compounds:
        f.write("%s\n" % item)

In [ ]:
omega = oeomega.OEOmega()
omega.SetStrictStereo(False) 

# Generate conformers then dock
inmols = []
usednames = []
for idx,line in enumerate(all_compounds):
    tmp = line.split()
    smi = tmp[0]
    mol = oechem.OEMol()
    name = tmp[1]
    if name=='' or name==None or len(name)<3:
        #Define alternate name based on index
        name = 'mol%s smiles %s' % (idx, smi)
        print("No name found on line %s; using alternate name %s..." % (idx, name))
    if not name in usednames: 
        usednames.append(name)
        oechem.OEParseSmiles(mol, smi)
        mol.SetTitle(name)
        builtOK = omega(mol)
        inmols.append(mol)
    else:
        continue

# Define how many docked poses to generate per molecule
num_poses = 2


# Open a filestream to write the docked poses
scores = {}
with oechem.oemolostream( 'dock-results-Chemscore.sdf') as ofs:

    # Loop over 3D molecules from the input filestream
    for mcmol in inmols:

        # Call docking function
        dockedMol, score = dock_molecule( dock, sdtag, num_poses, mcmol )
        print("{} {} score = {:.4f}".format(sdtag, dockedMol.GetTitle(), score))

        # Write docked molecules to output filestream
        oechem.OEWriteMolecule(ofs, dockedMol)
        
        # Store scores
        scores[ mcmol.GetTitle()] = score

In [ ]:
active_smiles_by_name = {}
file = open('mobley-minh-actives', 'r') 
text = file.readlines()
file.close()
for line in text:
    tmp = line.split()
    active_smiles_by_name[tmp[1]] = tmp[0]

# Build list of titles sorted by score
sorted_titles = list(scores.keys())
sorted_titles.sort( key = lambda title: scores[title] )

# Count how many actives are found at which ranks
ct = 0
fnd_actives = []
for active_name in active_smiles_by_name.keys():
    if active_name in sorted_titles:
        ct += 1
        print("Active %s found in docking results at rank %s" % ( active_name, sorted_titles.index(active_name)))
        fnd_actives.append( active_name )

print("Total compounds: %s" % len(sorted_titles))

#Find number of actives
n_actives = len(fnd_actives)